In [ ]:
#this code with the modified sampeling period case 4 considering time delay at output
import gym
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pickle
import csv
from matplotlib.animation import FuncAnimation

# problem = "Pendulum-v1"
# env = gym.make(problem)

# num_states = env.observation_space.shape[0]
# print("Size of State Space ->  {}".format(num_states))
# num_actions = env.action_space.shape[0]
# print("Size of Action Space ->  {}".format(num_actions))

# upper_bound = env.action_space.high[0]
# lower_bound = env.action_space.low[0]

# print("Max Value of Action ->  {}".format(upper_bound))
# print("Min Value of Action ->  {}".format(lower_bound))

### Define Model

In [ ]:
### Setup the environment

num_states=13 # 3: Considering z pos, z vel, z acc; 4: pos,vel,acc,u ; 8: pos, vel, acc, p_pos, p_vel, p_acc, u, p_u

num_actions=1 # just controling u_z

num_measurements=3 # For time delayed systems

mass=0.67 #in kg

g=9.81
#upper_bound=5
#lower_bound=-5
upper_bound= mass*g # i.e. coefficient of thrust to weight=2

lower_bound= -mass * g # we cannot do better without upside-down

output_length=12

vel_reward_weight_factor=0.1

# lib_interface stuff
import glob
import ctypes
import math

libfile1 = glob.glob('../Sim_intfc/out/build/linux-default/libinterface_simulink.so.1')[0]
mylib1 = ctypes.CDLL(libfile1)

libfile2 = glob.glob('../Sim_intfc/out/build/linux-default/libinterface_simulink.so (copy).1')[0]
mylib2 = ctypes.CDLL(libfile2)

libfile3 = glob.glob('../Sim_intfc/out/build/linux-default/libinterface_simulink.so (another copy).1')[0]
mylib3 = ctypes.CDLL(libfile3)

libfile4 = glob.glob('../Sim_intfc/out/build/linux-default/libinterface_simulink.so (3rd copy).1')[0]
mylib4 = ctypes.CDLL(libfile4)

class EnvUAV_cmd_z:
    simulink_input_length=1

    simulink_states_length=28
    print_period=1 #in seconds
    episode_timeout=10 #in seconds
    terminal_states=[0,0,0]
    termination_causes={'reached_goal':0,'time_out':1,'nan_error':2}
    def __init__(self,simulation_dt,sampling_dt,termination_cost,mylib,mass=1,CTW=2,g=9.81):
        self.mylib = mylib
        self.mylib.LoadModel()
        self.mylib.run_a_step.restype = np.ctypeslib.ndpointer(dtype=np.float64,shape=(self.simulink_states_length,))
        self.mylib.run_a_step.argtypes = [np.ctypeslib.ndpointer(dtype=np.float64),ctypes.c_int]
        #self.mass=mass
        #self.CTW=CTW
        #self.g=g

        self.simulation_dt=simulation_dt
        self.sampling_dt=sampling_dt
        self.sim_steps_per_samp_period=round(self.sampling_dt/self.simulation_dt) # Note sim dt must not be larger than sampling dt

        self.steps_per_print_period=int(EnvUAV_cmd_z.print_period/self.sampling_dt)
        self.sampling_step_counter=0
        self.reset()
    
    def __call__(self,input):
        #print("norhan")
        self.sampling_step_counter=self.sampling_step_counter+1
        inputs = np.zeros(EnvUAV_cmd_z.simulink_input_length,dtype=np.float64)
        inputs[0]=np.copy(input[0])
        for i in range(self.sim_steps_per_samp_period): # Simulink solver is run at a higher rate for numerical stability
            self.states_meas = self.mylib.run_a_step(inputs,len(inputs))
        #terminal_states=[self.states_meas[25],0,0]   
        self.z_states_meas[0]=np.copy(self.states_meas[2])-self.states_meas[25]
        #print(self.z_states_meas[0])
        self.z_states_meas[1]=np.copy(self.states_meas[5])-self.states_meas[26]
        if num_states>2:
            self.z_states_meas[2]=np.copy(self.states_meas[8])-self.states_meas[27]
        if num_states==4:
            self.z_states_meas[3]=np.copy(input[0])
            self.prev_input=np.copy(input[0])
            self.prev_measurements=np.copy(self.z_states_meas[0:3])
        if num_states==7: #shehata suggestion d=4
            #self.z_states_meas[3:39]=np.copy(self.prevv_measurements)
            self.z_states_meas[3]=np.copy(input[0])
            self.z_states_meas[4:13]=np.copy(self.prevv_input)
            self.prevv_input[1:3] = self.prevv_input[0:2]
            self.prevv_input[0]=np.copy(input[0])
        if num_states==9: #shehata suggestion d=6
            #self.z_states_meas[3:39]=np.copy(self.prevv_measurements)
            self.z_states_meas[3]=np.copy(input[0])
            self.z_states_meas[4:9]=np.copy(self.prevv_input)
            self.prevv_input[1:8] = self.prevv_input[0:7]
            self.prevv_input[0]=np.copy(input[0])
        if num_states==19: #d=4
            self.z_states_meas[3:15]=np.copy(self.prevv_measurements)
            self.z_states_meas[15]=np.copy(input[0])
            self.z_states_meas[16:19]=np.copy(self.prevv_input)
            self.prevv_input[1:3] = self.prevv_input[0:2]
            self.prevv_input[0]=np.copy(input[0])
            self.prevv_measurements=np.copy(self.z_states_meas[0:12])
            #print(self.z_states_meas)
            
        if num_states==23: #d=5
            self.z_states_meas[3:18]=np.copy(self.prevv_measurements)
            self.z_states_meas[18]=np.copy(input[0])
            self.z_states_meas[19:23]=np.copy(self.prevv_input)
            self.prevv_input[1:4] = self.prevv_input[0:3]
            self.prevv_input[0]=np.copy(input[0])
            self.prevv_measurements=np.copy(self.z_states_meas[0:15])
            
        if num_states==27: #d=6
            self.z_states_meas[3:21]=np.copy(self.prevv_measurements)
            self.z_states_meas[21]=np.copy(input[0])
            self.z_states_meas[22:27]=np.copy(self.prevv_input)
            self.prevv_input[1:5] = self.prevv_input[0:4]
            self.prevv_input[0]=np.copy(input[0])
            self.prevv_measurements=np.copy(self.z_states_meas[0:18])
        if num_states==43: #d=10
            self.z_states_meas[3:33]=np.copy(self.prevv_measurements)
            self.z_states_meas[33]=np.copy(input[0])
            self.z_states_meas[34:43]=np.copy(self.prevv_input)
            self.prevv_input[1:9] = self.prevv_input[0:8]
            self.prevv_input[0]=np.copy(input[0])
            self.prevv_measurements=np.copy(self.z_states_meas[0:30])
        if num_states==15: #shehata suggestion
            #self.z_states_meas[3:39]=np.copy(self.prevv_measurements)
            self.z_states_meas[3]=np.copy(input[0])
            self.z_states_meas[4:15]=np.copy(self.prevv_input)
            self.prevv_input[1:11] = self.prevv_input[0:10]
            self.prevv_input[0]=np.copy(input[0])
            #self.prev_measurements=np.copy(self.z_states_meas[0:36])
            #print(self.z_states_meas)
        if num_states==13: #shehata suggestion d=10
            #self.z_states_meas[3:39]=np.copy(self.prevv_measurements)
            self.z_states_meas[3]=np.copy(input[0])
            self.z_states_meas[4:13]=np.copy(self.prevv_input)
            self.prevv_input[1:9] = self.prevv_input[0:8]
            self.prevv_input[0]=np.copy(input[0])

        #cost=np.linalg.norm(np.array([self.z_states_meas[0],self.z_states_meas[1],self.z_states_meas[2]]))
        #terminal_states=[self.states_meas[25],0,0]
        #print(self.states_meas[25])
        cost=np.linalg.norm(np.array([self.z_states_meas[0],vel_reward_weight_factor*self.z_states_meas[1]]))
        self.reward=-cost*self.sampling_dt*100 # By 100 to 25 up reward for training
        if en_goal_termination==True:    
            if (cost<termination_cost):
                self.termination_cause=EnvUAV_cmd_z.termination_causes['reached_goal']
                print('Reached Goal!')
                self.done=True
        #print(self.sampling_step_counter)
        #print(self.sampling_step_counter*self.sampling_dt>EnvUAV_cmd_z.episode_timeout)
        #print(self.sampling_step_counter*self.sampling_dt)
        if (self.sampling_step_counter*self.sampling_dt>EnvUAV_cmd_z.episode_timeout):
            self.termination_cause=EnvUAV_cmd_z.termination_causes['time_out']
            #print(self.sampling_step_counter,self.sampling_dt,EnvUAV_cmd_z.episode_timeout)
            self.done=True
        
        if (math.isnan(self.z_states_meas[0]) or math.isnan(self.z_states_meas[1])):
            self.z_states_meas=[0,0,0] # This is incorrect, but hopefully it is waived by stochastic learning
            self.reward=np.ones(1)*0.0
            self.termination_cause=EnvUAV_cmd_z.termination_causes['nan_error']
            print('Warning NaN termination')
            self.done=True
        return self.z_states_meas, self.reward, self.done, self.termination_cause, self.states_meas[25], self.states_meas[2]

    def reset(self):
        if self.sampling_step_counter>0:
            self.mylib.TermModel()
        self.sampling_step_counter=0
        self.mylib.LoadModel()
        self.mylib.InitModel()
        self.prev_measurements=np.zeros(num_measurements)
        self.prevv_measurements=np.zeros(18)
        self.done=False
        self.z_states_meas=np.zeros(num_states)
        self.prev_input=0.0
        self.prevv_input=np.zeros(9)
        self.termination_cause=EnvUAV_cmd_z.termination_causes['reached_goal']
        return self.z_states_meas

    def print_states_periodic(self):
        if self.sampling_step_counter % self.steps_per_print_period ==1:
            print('z pos: ' + str(self.states_meas[2]))
            print('z vel: ' + str(self.states_meas[5]))
            print('simulation time (s): ' + str(self.sampling_step_counter*self.sampling_dt))
            print('===================')

termination_cost=0.5 # e.g. 0.05 is 5cm , 5cm/s , 5cm/s^2
simulink_dt=0.001 # Simulink solver step time
sampling_dt=0.06 # The one that would be used experimentally. e.g. 0.005 corresponds to 200Hz
UAV_alt_ctrl=EnvUAV_cmd_z(simulink_dt,sampling_dt,termination_cost, mylib1)
UAV_alt_ctrl2=EnvUAV_cmd_z(simulink_dt,sampling_dt,termination_cost, mylib2)
UAV_alt_ctrl3=EnvUAV_cmd_z(simulink_dt,sampling_dt,termination_cost, mylib3)
UAV_alt_ctrl4=EnvUAV_cmd_z(simulink_dt,sampling_dt,termination_cost, mylib4)



In [ ]:
print(UAV_alt_ctrl.sim_steps_per_samp_period)

In [ ]:
class OUActionNoise:
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None):
        self.theta = theta
        self.mean = mean
        self.std_dev = std_deviation
        self.dt = dt
        self.x_initial = x_initial
        self.reset()

    def __call__(self):
        # Formula taken from https://www.wikipedia.org/wiki/Ornstein-Uhlenbeck_process.
        x = (
            self.x_prev
            + self.theta * (self.mean - self.x_prev) * self.dt
            + self.std_dev * np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
        )
        # Store x into x_prev
        # Makes next noise dependent on current one
        self.x_prev = x
        return x

    def reset(self):
        if self.x_initial is not None:
            self.x_prev = self.x_initial
        else:
            self.x_prev = np.zeros_like(self.mean)

In [ ]:
class Buffer:
    def __init__(self,target_actor,target_critic,actor_model, critic_model,gamma, critic_optimizerr,actor_optimizerr, buffer_capacity=100000, batch_size=64):
        # Number of "experiences" to store at max
        self.buffer_capacity = buffer_capacity
        # Num of tuples to train on.
        self.batch_size = batch_size
        self.critic_optimizerr = critic_optimizerr
        self.actor_optimizerr = actor_optimizerr
        # Its tells us num of times record() was called.
        self.buffer_counter = 0
        self.gamma = gamma
        self.target_actor = target_actor
        self.target_critic = target_critic
        self.actor_model = actor_model
        self.critic_model = critic_model
        # Instead of list of tuples as the exp.replay concept go
        # We use different np.arrays for each tuple element
        self.state_buffer = np.zeros((self.buffer_capacity, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, num_actions))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, num_states))

    # Takes (s,a,r,s') obervation tuple as input
    def record(self, obs_tuple):
        # Set index to zero if buffer_capacity is exceeded,
        # replacing old records
        index = self.buffer_counter % self.buffer_capacity

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]

        self.buffer_counter += 1

    # Eager execution is turned on by default in TensorFlow 2. Decorating with tf.function allows
    # TensorFlow to build a static graph out of the logic and computations in our function.
    # This provides a large speed up for blocks of code that contain many small TensorFlow operations such as this one.
    @tf.function
    def update(
        self, state_batch, action_batch, reward_batch, next_state_batch,
    ):
        # Training and updating Actor & Critic networks.
        # See Pseudo Code.
        #print("ready")
        with tf.GradientTape() as tape:
            #print("steady")
            target_actions = self.target_actor(next_state_batch, training=True)
            #print("steady0")
            y = reward_batch + self.gamma * self.target_critic(
                [next_state_batch, target_actions], training=True
            )
            #print("steady")
            critic_value = self.critic_model([state_batch, action_batch], training=True)
            #print("steady2")
            critic_loss = tf.math.reduce_mean(tf.math.square(y - critic_value))
        #print("steady")
        critic_grad = tape.gradient(critic_loss, self.critic_model.trainable_variables)
        #print(critic_grad)
        #print(self.critic_model.trainable_variables)
        #print("steadyyayay")
        #print(zip(critic_grad, self.critic_model.trainable_variables))
        self.critic_optimizerr.apply_gradients(zip(critic_grad, self.critic_model.trainable_variables))
        #print("steadyyay")
        with tf.GradientTape() as tape:
            actions = self.actor_model(state_batch, training=True)
            critic_value = self.critic_model([state_batch, actions], training=True)
            # Used `-value` as we want to maximize the value given
            # by the critic for our actions
            actor_loss = -tf.math.reduce_mean(critic_value)
        #print("go")
        actor_grad = tape.gradient(actor_loss, self.actor_model.trainable_variables)
        self.actor_optimizerr.apply_gradients(
            zip(actor_grad, self.actor_model.trainable_variables)
        )
        #print("yay")

    # We compute the loss and update parameters
    def learn(self):
        # Get sampling range
        #print("ready")
        record_range = min(self.buffer_counter, self.buffer_capacity)
        # Randomly sample indices
        #print("steady")
        batch_indices = np.random.choice(record_range, self.batch_size)
        
        # Convert to tensors
        #print("go")
        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])
        #print("yay")
        self.update(state_batch, action_batch, reward_batch, next_state_batch)
        #print("yay")

# This update target parameters slowly
# Based on rate `tau`, which is much less than one.
@tf.function
def update_target(target_weights, weights, tau=0.005):
    for (a, b) in zip(target_weights, weights):
        a.assign(b * 0.005 + a * (1 - 0.005))

### Define Actor-Critic

In [ ]:
def get_actor(scale_actor):
    # Initialize weights between -3e-3 and 3-e3
    last_init = tf.random_uniform_initializer(minval=-0.2, maxval=0.2) #MC

    inputs = layers.Input(shape=(num_states,))
    out = layers.Dense(scale_actor*512, activation="relu")(inputs)
    out = layers.Dense(scale_actor*512, activation="relu")(out)
    outputs = layers.Dense(1, activation="tanh", kernel_initializer=last_init)(out)

    # Our upper bound is 2.0 for Pendulum.
    outputs = outputs * upper_bound
    model = tf.keras.Model(inputs, outputs)
    return model


def get_critic(scale_critic):
    # State as input
    state_input = layers.Input(shape=(num_states))
    state_out = layers.Dense(scale_critic*32, activation="relu")(state_input)
    state_out = layers.Dense(scale_critic*64, activation="relu")(state_out)

    # Action as input
    action_input = layers.Input(shape=(num_actions))
    action_out = layers.Dense(scale_critic*64, activation="relu")(action_input)

    # Both are passed through seperate layer before concatenating
    concat = layers.Concatenate()([state_out, action_out])

    out = layers.Dense(scale_critic*512, activation="relu")(concat)
    out = layers.Dense(scale_critic*512, activation="relu")(out)
    outputs = layers.Dense(1)(out)

    # Outputs single value for give state-action
    model = tf.keras.Model([state_input, action_input], outputs)

    return model

In [ ]:
def policy(actor_model,state, noise_object):
    sampled_actions = tf.squeeze(actor_model(state))
    noise = noise_object()
    # Adding noise to action
    sampled_actions = sampled_actions.numpy() + noise # WARNING NOISE MIGHT BE DISABLED

    # We make sure action is within bounds
    legal_action = np.clip(sampled_actions, lower_bound, upper_bound)

    return [np.squeeze(legal_action)]

### Hyperparameters

In [ ]:
# RL models
actor_model = get_actor(1)
critic_model = get_critic(1)

target_actor = get_actor(1)
target_critic = get_critic(1)
#####
actor_model2 = get_actor(1)
critic_model2 = get_critic(1)

target_actor2 = get_actor(1)
target_critic2 = get_critic(1)
#####
actor_model3 = get_actor(1)
critic_model3 = get_critic(1)

target_actor3 = get_actor(1)
target_critic3 = get_critic(1)
#####
actor_model4 = get_actor(1)
critic_model4 = get_critic(1)

target_actor4 = get_actor(1)
target_critic4 = get_critic(1)
# Making the weights equal initially
target_actor.set_weights(actor_model.get_weights())
target_critic.set_weights(critic_model.get_weights())

target_actor2.set_weights(actor_model2.get_weights())
target_critic2.set_weights(critic_model2.get_weights())

target_actor3.set_weights(actor_model3.get_weights())
target_critic3.set_weights(critic_model3.get_weights())

target_actor4.set_weights(actor_model4.get_weights())
target_critic4.set_weights(critic_model4.get_weights())

# Learning rate for actor-critic models
critic_lr = 0.002
actor_lr = 0.001
####
critic_lr2 = 0.002
actor_lr2 = 0.001
####
critic_lr3 = 0.002
actor_lr3 = 0.001
####
critic_lr4 = 0.002
actor_lr4 = 0.001

critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
actor_optimizer = tf.keras.optimizers.Adam(actor_lr)
#####
critic_optimizer2 = tf.keras.optimizers.Adam(critic_lr2)
actor_optimizer2 = tf.keras.optimizers.Adam(actor_lr2)
#####
critic_optimizer3 = tf.keras.optimizers.Adam(critic_lr3)
actor_optimizer3 = tf.keras.optimizers.Adam(actor_lr3)
#####
critic_optimizer4 = tf.keras.optimizers.Adam(critic_lr4)
actor_optimizer4 = tf.keras.optimizers.Adam(actor_lr4)

total_episodes = 5000
# Discount factor for future rewards
gamma = 0.99
# Used to update target networks
tau = 0.005

buffer = Buffer(target_actor,target_critic,actor_model, critic_model, gamma, critic_optimizer,actor_optimizer,200000, 1024)
##
buffer2 = Buffer(target_actor2,target_critic2,actor_model2, critic_model2, gamma, critic_optimizer2,actor_optimizer2,200000, 1024)
##
buffer3 = Buffer(target_actor3,target_critic3,actor_model3, critic_model3, gamma, critic_optimizer3,actor_optimizer3,200000, 1024)
##
buffer4 = Buffer(target_actor4,target_critic4,actor_model4, critic_model4, gamma, critic_optimizer4,actor_optimizer4,200000, 1024)


# nosie model
init_std_dev = 0.4
final_exponent_val=10 #formula std_dev=init_std_dev * exp^(-(ep/total_episodes)*final_exponent_val)
ou_noise = OUActionNoise(mean=np.zeros(1), std_deviation=float(init_std_dev) * np.ones(1))
en_noise_decay=False

# model behaviour
en_goal_termination=False

train_inputs = [
        (UAV_alt_ctrl, buffer, actor_model, critic_model, target_actor, target_critic, critic_lr, actor_lr),
        #(UAV_alt_ctrl2, buffer2, actor_model2, critic_model2, target_actor2, target_critic2, critic_lr2, actor_lr2),
        #(UAV_alt_ctrl3, buffer3, actor_model3, critic_model3, target_actor3, target_critic3, critic_lr3, actor_lr3),
        #(UAV_alt_ctrl4, buffer4, actor_model4, critic_model4, target_actor4, target_critic4, critic_lr4, actor_lr4)
    ]


### Training function definition

In [ ]:
def train(indexx, UAV_alt_ctrll, bufferr, actor_modell, critic_modell, target_actorr, target_criticc, critic_lrr, actor_lrr):
    # To store reward history of each episode
    ep_reward_list = []

    # To store average reward history of the last few episodes
    avg_reward_list = []

    # To store final states of the system
    final_states_reward_list = []  # [states, reward]

    # To store termination status
    termination_causes_list = []

    # To store all state_action pairs
    states_action_list = []
    states_action_lists = []
    window_state_list = []
    window_state_lists = []
    z_state_list = []
    z_state_lists = []

    learning_frequency = 256  # MC

    for ep in range(total_episodes):
        # prev_state = env.reset()
        prev_state = UAV_alt_ctrll.reset()
        episodic_reward = 0  # Total rewards in one episode
        #print(prev_state)
        while True:
            # Uncomment this to see the Actor in action
            # But not in a python notebook.
            # env.render()
            tf_prev_state = tf.expand_dims(tf.convert_to_tensor(prev_state), 0)  # Necessary for tensorflow (check immutable/mutable data types if you want to know more)
            #print(tf_prev_state)
            action = policy(actor_modell,tf_prev_state, ou_noise)
            # Receive state and reward from the environment.
            # state, reward, done, info = env.step(action)
            #print("before execution")
            #print(action)
            state, reward, done, termination_cause, window_state, z_position = UAV_alt_ctrll(action)
            #print("after execution")
            states_action_list.append([state.copy(), action[0].copy()])
            #print("1")
            window_state_list.append(window_state.copy())
            #print("2")
            z_state_list.append(z_position.copy())
            #print("3")
            # UAV_alt_ctrl.print_states_periodic()

            buffer.record((prev_state.copy(), action.copy(), reward.copy(), state.copy()))
            episodic_reward += reward
            #print("4")
            if buffer.buffer_counter > buffer.batch_size:
                #print("6")
                if (buffer.buffer_counter % (int(buffer.batch_size / learning_frequency)) == 1):
                    #print("7raaaaaaaam")
                    buffer.learn()
                    print("learn")
                    update_target(target_actorr.variables, actor_modell.variables, 0.005)
                    update_target(target_criticc.variables, critic_modell.variables, 0.005)
            #print("5")
            # End this episode when `done` is True
            
            #print("before done")
            if done:
                break

            prev_state = np.copy(state)
            #print(prev_state)
            
        #print("outside while")
        ep_reward_list.append(episodic_reward)
        termination_causes_list.append(termination_cause)
        final_states_reward_list.append([state.copy(), reward.copy()])
        states_action_lists.append(states_action_list.copy())
        window_state_lists.append(window_state_list.copy())
        z_state_lists.append(z_state_list.copy())
        states_action_list.clear()
        window_state_list.clear()
        z_state_list.clear()

        # Mean of the last 40 episodes
        avg_reward = np.mean(ep_reward_list[-40:])
        print("Episode * {} * Avg Reward is ==> {} from process {}".format(ep, avg_reward,indexx))
        print("Episode * {} * Reward is ==> {}".format(ep, episodic_reward))

        avg_reward_list.append(avg_reward)
        if en_noise_decay:
            ou_noise.std_dev[0] = init_std_dev * math.exp(-(ep / total_episodes) * final_exponent_val)
    return termination_causes_list, z_state_lists, window_state_lists, states_action_lists, ep_reward_list, avg_reward_list



### Train

In [ ]:
#termination_causes_list, z_state_lists, window_state_lists, states_action_lists, ep_reward_list, avg_reward_list = train(1, UAV_alt_ctrl, buffer, actor_model, critic_model, target_actor, target_critic, critic_lr, actor_lr)
import tensorflow as tf
import numpy as np
import multiprocessing
from contextlib import redirect_stdout
import io

class TrainingProcess(multiprocessing.Process):
    def __init__(self, input_args, results_queue):
        super().__init__()
        self.input_args = input_args
        self.results_queue = results_queue

    def run(self):
        indexx, UAV_alt_ctrl, buffer, actor_model, critic_model, target_actor, target_critic, critic_lr, actor_lr = self.input_args

        # Create a separate StringIO for capturing stdout
        stdout_capture = io.StringIO()

        #with redirect_stdout(stdout_capture):
        termination_causes_list, z_state_lists, window_state_lists, states_action_lists, ep_reward_list, avg_reward_list = train(indexx,UAV_alt_ctrl, buffer, actor_model, critic_model, target_actor, target_critic, critic_lr, actor_lr)

            
        # Store the results in a tuple
        results = (termination_causes_list, z_state_lists, window_state_lists, states_action_lists, ep_reward_list, avg_reward_list)  # Corrected to use the calculated result
            
        # Put the results into the queue
        self.results_queue.put(results)    
            
            
results_queue = multiprocessing.Queue()

# Create and start separate processes for each training
processes = []
for i, input_args in enumerate(train_inputs):
    input_args = (i,) + input_args
    process = TrainingProcess(input_args, results_queue)
    processes.append(process)
    print(f"Starting process: {i}")
    #print("norhan")
    process.start()
    #print("norhan")
#print("norhan")
# # Wait for all processes to finish
for process in processes:
    process.join()

print("processes joined")
# Collect results from the queue
results_list = [results_queue.get() for _ in range(len(train_inputs))]
# Access the results from the results_list for each process
process1_results = results_list[0]
process2_results = results_list[1]
process3_results = results_list[2]
process4_results = results_list[3]

termination_causes_list, z_state_lists, window_state_lists, states_action_lists, ep_reward_list, avg_reward_list = process1_results
termination_causes_list_2, z_state_lists_2, window_state_lists_2, states_action_lists_2, ep_reward_list_2, avg_reward_list_2 = process2_results
termination_causes_list_3, z_state_lists_3, window_state_lists_3, states_action_lists_3, ep_reward_list_3, avg_reward_list_3 = process3_results
termination_causes_list_4, z_state_lists_4, window_state_lists_4, states_action_lists_4, ep_reward_list_4, avg_reward_list_4 = process4_results




In [ ]:
#print(termination_causes_list)
print(processes)
print(termination_causes_list,termination_causes_list_2,termination_causes_list_3,termination_causes_list_4)

In [ ]:
# Plotting graph
# Episodes versus Avg. Rewards
num_of_episodes=len(avg_reward_list)
t=np.arange(num_of_episodes)+(len(avg_reward_list)-num_of_episodes)
plt.plot(t,avg_reward_list[-num_of_episodes:])
plt.xlabel("Episode")
plt.ylabel("Avg. Epsiodic Reward")
plt.show()
plt.plot(t,ep_reward_list[-num_of_episodes:])
plt.xlabel("Episode")
plt.ylabel("Epsiodic Reward")
plt.show()


In [ ]:
# Plotting states
index_of_episode=150
pos_vals=[]
vel_vals=[]
acc_vals=[]
act_vals=[]
window_vals=[]
z_vals=[]
inst_reward=[]
accum_reward=[]
time_vals=np.linspace(0,EnvUAV_cmd_z.episode_timeout,len(states_action_lists[index_of_episode]))

for tuble in states_action_lists[index_of_episode]:
    pos_vals.append(tuble[0][0])
    vel_vals.append(tuble[0][1])
    acc_vals.append(tuble[0][2])
    act_vals.append(tuble[1])
    inst_reward.append(-np.linalg.norm(np.array([pos_vals[-1],vel_reward_weight_factor*vel_vals[-1]]))*UAV_alt_ctrl.sampling_dt*100*0.1)
    accum_reward.append(sum(inst_reward))

window_vals = window_state_lists[index_of_episode]
z_vals = z_state_lists[index_of_episode]

plt.close()
figure(figsize=(24, 10), dpi=80)

plt.plot(time_vals,pos_vals)
plt.plot(time_vals,vel_vals)
plt.plot(time_vals,acc_vals)
plt.plot(time_vals,act_vals)
#plt.plot(time_vals,accum_reward)
#plt.plot(time_vals,inst_reward)


plt.xlabel("Time")
plt.ylabel("State and Action Trajectories")
plt.rcParams.update({'font.size': 20})

plt.legend(['Position','Velocity','Acceleration','Actor','accumelated reward (x0.1)','Instantaneous reward'])
plt.grid()
plt.show()

time = time_vals
altitude = z_vals

# Sample window position vs. time data (replace with your data)
window_time = time_vals
window_position = window_vals

# Create a figure and axis for the plot
fig, ax = plt.subplots()
plt.plot(time_vals,altitude)
plt.plot(time_vals,window_position)
ax.set_xlim(0, max(max(time), max(window_time)))
ax.set_ylim(min(min(altitude), min(window_position)) - 1, max(max(altitude), max(window_position)) + 1)
plt.xlabel('Time')
plt.ylabel('Altitude')
plt.title('Drone Altitude and Window Altitude vs. Time')
plt.legend(['Drone Altitude','Window Altitude'])
plt.grid()
plt.show()

print(window_state_lists[index_of_episode])


In [ ]:



# Specify the CSV file name
csv_file = 'time_data.csv'

# Open the CSV file in write mode
with open(csv_file, 'w') as file:
    # Write the vector data to the file
    for value in time_vals:
        file.write(str(value) + '\n')

print(f"Vector data saved to {csv_file} successfully.")



In [ ]:
# Specify the CSV file name
csv_file = 'pos_data.csv'

# Open the CSV file in write mode
with open(csv_file, 'w') as file:
    # Write the vector data to the file
    for value in pos_vals:
        file.write(str(value) + '\n')

print(f"Vector data saved to {csv_file} successfully.")


In [ ]:
# Specify the CSV file name
csv_file = 'actor_data.csv'

# Open the CSV file in write mode
with open(csv_file, 'w') as file:
    # Write the vector data to the file
    for value in act_vals:
        file.write(str(value) + '\n')

print(f"Vector data saved to {csv_file} successfully.")


In [ ]:
# Plotting of value function
%matplotlib widget
from random import sample
from tabnanny import verbose
from mpl_toolkits.mplot3d import Axes3D

import pickle
import csv

## Inquiry actor
# Mesh initiation
samples_per_dim=20

pos_range=[-100,100]
vel_range=[-100,100]
acc_range=[-5,5]

pos_states=np.linspace(pos_range[0],pos_range[1],samples_per_dim)

vel_states=np.linspace(vel_range[0],vel_range[1],samples_per_dim)

acc_states=np.linspace(acc_range[0],acc_range[1],samples_per_dim)

value_function=np.linspace((pos_range[0],vel_range[0]),(pos_range[1],vel_range[1]),samples_per_dim)

actor_output=np.zeros([samples_per_dim,samples_per_dim,samples_per_dim])

value_function=np.ones([samples_per_dim,samples_per_dim,samples_per_dim])

default_acc_state=acc_states[-1]

for i in range(samples_per_dim):
    for j in range(samples_per_dim):
        #for k in range(samples_per_dim):
        input_to_actor=np.asarray([pos_states[i],vel_states[j],default_acc_state])
        input_to_actor=input_to_actor.reshape(1,3)
        actor_output[i,j,0]=actor_model.predict(input_to_actor,verbose=0)


for i in range(samples_per_dim):
    for j in range(samples_per_dim):
        #for k in range(samples_per_dim):
        input_to_critic_states=np.asarray([pos_states[i],vel_states[j],default_acc_state])
        input_to_critic_states=input_to_critic_states.reshape(1,3)  #norhan
        input_to_critic_action=np.asarray([actor_output[i,j,0]])
        value_function[i,j,0]=critic_model([input_to_critic_states,input_to_critic_action])
            




In [ ]:
# creating figure
fig = plt.figure()
ax = Axes3D(fig)
  
with open('value_function.csv','w') as f:
    write = csv.writer(f)
    for i in range(samples_per_dim):
        for j in range(samples_per_dim):
            ax.scatter(pos_states[i], vel_states[j], value_function[i,j,0], color='green')
            #write_list=[pos_states[i],vel_states[j],value_function[i,j,0]]
            #write.writerows(write_list)


  
# setting title and labels
ax.set_title("3D plot")
ax.set_xlabel('Position')
ax.set_ylabel('Velocity')
ax.set_zlabel('Value Function')
  
# displaying the plot
plt.show()

In [ ]:
# Plotting actor output

forced_acc_state=0.0
num_pts=30

actor_output=np.zeros(num_pts)

# 1 fix velocity and vary position
forced_velocity=0.0
pos_max=10

ranged_position=np.linspace(-pos_max,pos_max,num_pts)

for i in range(num_pts):
    input_to_actor=np.asarray([ranged_position[i],forced_velocity,forced_acc_state])
    input_to_actor=input_to_actor.reshape(1,3)
    actor_output[i]=actor_model.predict(input_to_actor,verbose=0)

plt.close()
fig=plt.figure(figsize=(24, 10), dpi=80)
axes=fig.add_subplot(1,1,1)
axes.set_xticks(np.arange(-pos_max,pos_max,1))
plt.plot(ranged_position,actor_output)
plt.xlabel("position")
plt.ylabel("action")


plt.grid()
plt.show()


In [ ]:
# Plotting actor output

# 2 fix position and vary velocity
forced_position=0.0
vel_max=10
ranged_vel=np.linspace(-vel_max,vel_max,num_pts)

for i in range(num_pts):
    input_to_actor=np.asarray([forced_position,ranged_vel[i],forced_acc_state])
    input_to_actor=input_to_actor.reshape(1,3)
    actor_output[i]=actor_model.predict(input_to_actor,verbose=0)

plt.close()
fig=plt.figure(figsize=(24, 10), dpi=80)
axes=fig.add_subplot(1,1,1)
axes.set_xticks(np.arange(-vel_max,vel_max,1))
plt.plot(ranged_vel,actor_output)
plt.xlabel("vel")
plt.ylabel("action")

plt.grid()

plt.show()

### Save

In [ ]:
# Prepared data for save in csv
import pickle
import csv

index_of_episode=7
flat_list=[]

for tuble in states_action_lists[index_of_episode]:
    flat_list.append([tuble[0].tolist()+[tuble[1].tolist()]][0])

with open('episode_trajectories.csv','w') as f:
    write = csv.writer(f)
    write.writerows(flat_list)


In [ ]:
# Save the weights
actor_model.save_weights("uav_z_u_cmd_actor.h5")
critic_model.save_weights("uav_z_u_cmd_critic.h5")
actor_model.save('model2',save_format='tf')

target_actor.save_weights("uav_z_u_cmd_target_actor.h5")
target_critic.save_weights("uav_z_u_cmd_target_critic.h5")


In [ ]:
from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2,
)

model = actor_model
# Create frozen graph
x = tf.TensorSpec(model.input_shape, tf.float32, name="x")
concrete_function = tf.function(lambda x: model(x)).get_concrete_function(x)
frozen_model = convert_variables_to_constants_v2(concrete_function)

# Check input/output node name
print(f"{frozen_model.inputs=}")
print(f"{frozen_model.outputs=}")

# Save the graph as protobuf format
directory = "."
tf.io.write_graph(frozen_model.graph, directory, "model3.pb", as_text=False)


In [ ]:

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(actor_model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)


In [ ]:

def print_to_csv(name,*argv):
    with open(name,'w') as f:
        write = csv.writer(f)
        write.writerows(argv)
        

### Continue Training

In [ ]:
# 


additional_training_episodes=400

# Takes about 4 min to train
for ep in range(additional_training_episodes):

    #prev_state = env.reset()
    prev_state = UAV_alt_ctrl.reset()
    episodic_reward = 0

    while True:
        # Uncomment this to see the Actor in action
        # But not in a python notebook.
        # env.render()

        

        tf_prev_state = tf.expand_dims(tf.convert_to_tensor(prev_state), 0)

        action = policy(tf_prev_state, ou_noise)
        # Recieve state and reward from environment.
        #state, reward, done, info = env.step(action)
        state, reward, done, termination_cause=UAV_alt_ctrl(action)
        states_action_list.append([state.copy(),action[0].copy()])
        #UAV_alt_ctrl.print_states_periodic()

        buffer.record((prev_state.copy(), action.copy(), reward.copy(), state.copy()))
        episodic_reward += reward

        if buffer.buffer_counter>buffer.batch_size:
            if (buffer.buffer_counter%(int(buffer.batch_size/learning_frequency))==1):
                buffer.learn()
                update_target(target_actor.variables, actor_model.variables, 0.005)
                update_target(target_critic.variables, critic_model.variables, 0.005)


        # End this episode when `done` is True
        if done:
            break

        prev_state = np.copy(state)

    ep_reward_list.append(episodic_reward)
    termination_causes_list.append(termination_cause)
    final_states_reward_list.append([state.copy(),reward.copy()])
    states_action_lists.append(states_action_list.copy())
    states_action_list.clear()

    # Mean of last 40 episodes
    avg_reward = np.mean(ep_reward_list[-40:])
    print("Episode * {} * Avg Reward is ==> {}".format(ep, avg_reward))
    print("Episode * {} * Reward is ==> {}".format(ep, episodic_reward))
    
    avg_reward_list.append(avg_reward)
    if en_noise_decay:
        ou_noise.std_dev[0]=init_std_dev * math.exp(-(ep/total_episodes)*final_exponent_val)



### Load

In [ ]:
loaded_actor_model = get_actor()

loaded_critic_model=get_critic()

loaded_actor_model.load_weights("uav_z_u_cmd_actor.h5")

loaded_critic_model.load_weights("uav_z_u_cmd_critic.h5")

### Run Actor

In [ ]:

en_run_loaded_actor=True
en_accelerometer_noise=False
acc_noise_std_dev=1

test_UAV_alt_ctrl=EnvUAV_cmd_z(simulink_dt,sampling_dt,termination_cost)

test_time=5.0 # Seconds

running_steps=int(test_time/sampling_dt)
states=np.zeros(num_states)
states=test_UAV_alt_ctrl.reset()

states_history=np.zeros([running_steps,num_states])
actions_history=np.zeros(running_steps)
for i in range(running_steps):
    if en_accelerometer_noise:
        states[2]=states[2]+np.random.normal(0.0,acc_noise_std_dev)
    input_to_actor=np.asarray(states)
    input_to_actor=input_to_actor.reshape(1,num_states)
    if en_run_loaded_actor:
        actor_output=loaded_actor_model.predict(input_to_actor,verbose=0)
    else:
        actor_output=actor_model.predict(input_to_actor,verbose=0)
    
    states=test_UAV_alt_ctrl(actor_output)
    states=states[0]
    states_history[i]=np.copy(states)
    actions_history[i]=np.copy(actor_output)

time_vals=np.linspace(0,test_time,running_steps)


In [ ]:
#plot
plt_range_time=test_time
plotting_range=int((plt_range_time/test_time)*running_steps)


plt.close()

fig=plt.figure(figsize=(3, 5), dpi=160)
axes=fig.add_subplot(1,1,1)
axes.set_xticks(np.arange(0,test_time,1))

plt.plot(time_vals[0:plotting_range],states_history[0:plotting_range,0], linewidth=0.75, label=r'Position ($m$)')
plt.plot(time_vals[0:plotting_range],states_history[0:plotting_range,1], linewidth=0.75, label=r'Velocity ($\frac{m}{s}$)')
plt.plot(time_vals[0:plotting_range],states_history[0:plotting_range,2], linewidth=0.75, label=r'Acceleration ($\frac{m}{s^2}$)')
plt.plot(time_vals[0:plotting_range],actions_history[0:plotting_range], linewidth=0.75, label=r'Actor')

plt.xlabel("Time (s)")
plt.ylabel("State Trajectories")
plt.legend(loc=4)
plt.xlim([0, plt_range_time])

plt.grid()
plt.show()
print(states_history)
#print_to_csv('hi.csv',states_history[:,0],states_history[:,1],states_history[:,2])

### Debugging

In [ ]:
# Testing noise actions
noise_tester=OUActionNoise(mean=np.zeros(1),std_deviation=np.ones(1)*0.04)

list_generated_noise=[]


for i in range(1000):
    list_generated_noise.append(noise_tester())


plt.close()

plt.plot(list_generated_noise)

plt.show()